# Import from each file

In [1]:
from creditcard_ensemble_bag import best_model as bag_model # 30 seconds to 1 minute, 30 seconds
from creditcard_ensemble_RFC import best_model as RFC_model # 30 seconds to 1 minute
from creditcard_ensemble_AB import best_model as AB_model # 6-10 minutes
from creditcard_ensemble_GB import best_model as GB_model # 30 minutes
from creditcard_ensemble_HGB import best_model as HGB_model # 30 seconds
from creditcard_ensemble_SC import best_model as stack_model # 20 minutes
from creditcard_ensemble_voting import best_model as voting_model # 10 minutes


import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score


from creditcard_preparation import prepare_creditcard_data, combine_algo_and_pipeline

In [2]:
def new_eval(algo, X_train, y_train, X_test, y_test):

    try:

        algo.fit(X_train,np.ravel(y_train))

        test_predictions = algo.predict(X_test)

        accuracy = accuracy_score(y_test, test_predictions)

        scores = {"Accuracy": accuracy}

    except:

        # alternate data (no raveling performed)

        algo.fit(X_train,y_train)

        test_predictions = algo.predict(X_test)
    
        accuracy = accuracy_score(y_test, test_predictions)

        scores = {"Accuracy": accuracy}

        

    return scores

# Run Ensemble Models

In [3]:
# Run all models and combine results into dataframe

X_train, X_dev, X_test, y_train, y_dev, y_test = prepare_creditcard_data((1/10,1/10))

print("Data Loaded")

models = [bag_model,
          RFC_model,
          AB_model,
          GB_model,
          HGB_model,
          voting_model,
          stack_model]

data = []

for model in models:

    print("Testing the", model)

    model_prepared = combine_algo_and_pipeline(model)

    data.append(new_eval(model_prepared, X_train, y_train, X_test, y_test))


scores = pd.DataFrame(data=tuple(data),
                     index=["BaggingClassifier",
                            "RandomForestClassifier",
                            "AdaBoostClassifier",
                            "GradientBoostingClassifier",
                            "HistGradientBoostingClassifier",
                            "VotingClassifier",
                            "StackingClassifier"]
                     ).sort_values("Accuracy", ascending=False)
scores

Data Loaded
Testing the BaggingClassifier(estimator=DecisionTreeClassifier(), max_features=0.7,
                  max_samples=0.7)
Testing the RandomForestClassifier(min_samples_leaf=4, min_samples_split=5, n_estimators=10)
Testing the AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=10,
                                                    max_features=0.8,
                                                    min_samples_leaf=10,
                                                    random_state=42))


/Users/cody/Documents/DS420/Project2/proj2/.venv/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Testing the GradientBoostingClassifier(max_depth=10, max_features=0.8, min_samples_leaf=10,
                           n_estimators=500, n_iter_no_change=2,
                           random_state=42, subsample=0.9)
Testing the HistGradientBoostingClassifier(l2_regularization=0.2, learning_rate=1.0)
Testing the VotingClassifier(estimators=[('dt', DecisionTreeClassifier()),
                             ('lr', LogisticRegression()),
                             ('rfc', RandomForestClassifier())],
                 n_jobs=-1)
Testing the StackingClassifier(cv=5,
                   estimators=[('RF',
                                RandomForestClassifier(min_samples_leaf=4,
                                                       min_samples_split=5,
                                                       n_estimators=10)),
                               ('LR', LogisticRegression())],
                   final_estimator=DecisionTreeClassifier(criterion='entropy'),
                   n_jobs=-1)


,Accuracy
AdaBoostClassifier,0.999859
StackingClassifier,0.999807
GradientBoostingClassifier,0.999807
RandomForestClassifier,0.999719
BaggingClassifier,0.999666
VotingClassifier,0.999086
HistGradientBoostingClassifier,0.992192
